# Creating a Migration Algorithm

This tutorial demonstrates how we can create a simple migration algorithm on EdgeSimPy.

First, let's import the EdgeSimPy modules:

In [1]:
# Downloading EdgeSimPy binaries from GitHub (the "-q" parameter suppresses Pip's output. You check the full logs by removing it)
# %pip install -q git+https://github.com/EdgeSimPy/EdgeSimPy.git@v1.1.0

# Importing EdgeSimPy components
from edge_sim_py import *

## Implementing the Migration Algorithm

Our simple migration algorithm will work according to the well-known Worst-Fit heuristic. In a nutshell, it will provision each service to the edge server with the largest amount of free resources.

In [2]:
def my_algorithm(parameters):
    # Let's iterate over the list of services using the 'all()' helper method
    print("\n\n")
    for service in Service.all():
        # We don't want to migrate services are are already being migrated
        if not service.being_provisioned:
            # We need to sort edge servers based on amount of free resources they have. To do so, we are going to use Python's
            # "sorted" method (you can learn more about "sorted()" in this link: https://docs.python.org/3/howto/sorting.html). As
            # the capacity of edge servers is modeled in three layers (CPU, memory, and disk), we calculate the geometric mean
            # between these to get the average resource utilization of edge servers. Finally, we set the sorted method "reverse"
            # attribute as "True" as we want to sort edge servers by their free resources in descending order
            edge_servers = sorted(
                EdgeServer.all(),
                key=lambda s: ((s.cpu - s.cpu_demand) * (s.memory - s.memory_demand) * (s.disk - s.disk_demand)) ** (1 / 3),
                reverse=True,
            )
            for edge_server in edge_servers:
                # Checking if the edge server has resources to host the service
                if edge_server.has_capacity_to_host(service=service):
                    # We just need to migrate the service if it's not already in the least occupied edge server
                    if service.server != edge_server:
                        print(f"[STEP {parameters['current_step']}] Migrating {service} From {service.server} to {edge_server}")
                        service.provision(target_server=edge_server)
                        # After start migrating the service we can move on to the next service
                        break

## Running the Simulation

Before testing our algorithm, we must tell EdgeSimPy when it must stop the simulation. For example, let's run the simulation for 600 seconds (i.e., 10 minutes).

To do so, we must create a simple function that will be used as the simulation's stopping criterion. Behind the scenes, at the end of each time step, EdgeSimPy will run that function, halting the simulation if it returns `True`.

In [3]:
def stopping_criterion(model: object):
    # As EdgeSimPy will halt the simulation whenever this function returns True,
    # its output will be a boolean expression that checks if the current time step is 600
    return model.schedule.steps == 600

Once we have developed our stopping criterion function, we can create an instance of the `Simulation` class passing a couple of arguments, load a dataset, and run the simulation to check how our algorithm goes.

In [4]:
# Creating a Simulator object
simulator = Simulator(
    tick_duration=1,
    tick_unit="seconds",
    stopping_criterion=stopping_criterion,
    resource_management_algorithm=my_algorithm,
)

# Loading a sample dataset from GitHub
simulator.initialize(input_file="https://raw.githubusercontent.com/EdgeSimPy/edgesimpy-tutorials/master/datasets/sample_dataset1.json")

# Executing the simulation
simulator.run_model()




[STEP 1] Migrating Service_1 From EdgeServer_5 to EdgeServer_1
[STEP 1] Migrating Service_2 From EdgeServer_5 to EdgeServer_2
[STEP 1] Migrating Service_3 From EdgeServer_6 to EdgeServer_5
[STEP 1] Migrating Service_4 From EdgeServer_4 to EdgeServer_6
[STEP 1] Migrating Service_5 From EdgeServer_4 to EdgeServer_1
[STEP 1] Migrating Service_6 From EdgeServer_6 to EdgeServer_2



[STEP 2] Migrating Service_3 From EdgeServer_5 to EdgeServer_6
[STEP 2] Migrating Service_4 From EdgeServer_6 to EdgeServer_1



[STEP 3] Migrating Service_1 From EdgeServer_1 to EdgeServer_5
[STEP 3] Migrating Service_2 From EdgeServer_2 to EdgeServer_5
[STEP 3] Migrating Service_3 From EdgeServer_6 to EdgeServer_5



[STEP 4] Migrating Service_1 From EdgeServer_5 to EdgeServer_6
[STEP 4] Migrating Service_2 From EdgeServer_5 to EdgeServer_2
[STEP 4] Migrating Service_3 From EdgeServer_5 to EdgeServer_6



[STEP 5] Migrating Service_2 From EdgeServer_2 to EdgeServer_5
[STEP 5] Migrating Service_3 From EdgeSe